In [2]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn  as sns
sns.set();

In [3]:
coffee_df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-07-07/coffee_ratings.csv')
coffee_df

,total_cup_points,species,owner,country_of_origin,farm_name,lot_number,mill,ico_number,company,altitude,...,color,category_two_defects,expiration,certification_body,certification_address,certification_contact,unit_of_measurement,altitude_low_meters,altitude_high_meters,altitude_mean_meters
0,90.58,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,0,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
1,89.92,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,1,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
2,89.75,Arabica,grounds for health admin,Guatemala,"san marcos barrancas ""san cristobal cuch",NaN,NaN,NaN,NaN,1600 - 1800 m,...,NaN,0,"May 31st, 2011",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,1600.0,1800.0,1700.0
3,89.00,Arabica,yidnekachew dabessa,Ethiopia,yidnekachew dabessa coffee plantation,NaN,wolensu,NaN,yidnekachew debessa coffee plantation,1800-2200,...,Green,2,"March 25th, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1800.0,2200.0,2000.0
4,88.83,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,2,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,78.75,Robusta,luis robles,Ecuador,robustasa,Lavado 1,our own lab,NaN,robustasa,NaN,...,Blue-Green,1,"January 18th, 2017",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,NaN,NaN,NaN
1335,78.08,Robusta,luis robles,Ecuador,robustasa,Lavado 3,own laboratory,NaN,robustasa,40,...,Blue-Green,0,"January 18th, 2017",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,40.0,40.0,40.0
1336,77.17,Robusta,james moore,United States,fazenda cazengo,NaN,cafe cazengo,NaN,global opportunity fund,795 meters,...,NaN,6,"December 23rd, 2015",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,795.0,795.0,795.0
1337,75.08,Robusta,cafe politico,India,NaN,NaN,NaN,14-1118-2014-0087,cafe politico,NaN,...,Green,1,"August 25th, 2015",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,NaN,NaN,NaN


In [4]:
#Разделяем данные по колонке bag_weight на кол-во и единицы измерения массы
coffee_df[['Amount1', 'Weight']] = coffee_df['bag_weight'].str.extract(r'(\d+)(\D+)')
coffee_df[['Amount1', 'Weight']]

,Amount1,Weight
0,60,kg
1,60,kg
2,NaN,NaN
3,60,kg
4,60,kg
...,...,...
1334,2,kg
1335,2,kg
1336,1,kg
1337,5,lbs


In [5]:
#Преобразовуем в числовой тип
coffee_df['Amount1'] = pd.to_numeric(coffee_df['Amount1'],errors = 'coerce')

In [6]:
#Посчитаем вес одного мешка в кг для партии экспорта
exchange_rate = {'kg': 1, 'kg,lbs': 1, 'lbs': 2.20462}
s1 = coffee_df['Weight'].replace(exchange_rate, regex=True)
s2 = coffee_df['Amount1']
coffee_df['Amount_weight']  = s2 * s1

In [48]:
#Вес мешка не может быть 0, возьмем как 1
coffee_df['Amount_weight'].fillna(1)

0       60.0000
1       60.0000
2        1.0000
3       60.0000
4       60.0000
         ...   
1334     2.0000
1335     2.0000
1336     1.0000
1337    11.0231
1338    11.0231
Name: Amount_weight, Length: 1339, dtype: float64

In [11]:
#Вес партии экспорта в кг
coffee_df['most_export'] = (coffee_df['Amount_weight']  * coffee_df['number_of_bags']).round(2)
coffee_df['most_export'].fillna(1)

0       18000.00
1       18000.00
2           1.00
3       19200.00
4       18000.00
          ...   
1334        2.00
1335        2.00
1336        1.00
1337       11.02
1338       11.02
Name: most_export, Length: 1339, dtype: float64

In [52]:
most_import =coffee_df.groupby(['country_of_origin']).most_export.mean().to_frame()
most_import.sort_values(by='most_export', ascending=False).head(15)

,most_export
country_of_origin,
Ethiopia,409222.935806
Kenya,372770.040000
Costa Rica,315406.893725
Peru,280314.419000
Uganda,113355.000000
Honduras,106393.849057
Burundi,15600.000000
Nicaragua,14261.940870
Colombia,14047.039563
